In [1]:
import matplotlib.pyplot as plt
import pandas
import re
from pathlib import Path
import numpy as np
import random
import os

import Utilities

In [2]:
def SetCPList(cp_num):
    CPlist_NYC = Utilities.GetScenarioCSVList(cp_num, True, "NYC")
    CPlist_ITH = Utilities.GetScenarioCSVList(cp_num, True, "ITH")
    CPlist_ISR = Utilities.GetScenarioCSVList(cp_num, True, "ISR")
    return CPlist_NYC, CPlist_ITH, CPlist_ISR

In [5]:
def FirstOutAnalysis(CPlist, carA_out_direction, carB_out_direction, out_threshold):
    timeA_list = []
    timeB_list = []

    for cp in CPlist:
        
        cwd = os.getcwd()
        cp = Path(cwd) / cp

        dfA, dfB = Utilities.GetTruncatedScenario(cp, out_threshold)
        
        # data doesn't have time column, so we need to create one
        dfA['Time'] = np.arange(len(dfA))  # arange returns evenly spaced values within a given interval
        dfB['Time'] = np.arange(len(dfB))
        

        timeA, timeB = 99999,99999

        if carA_out_direction == "top":
            # check if carA's Z position is ever greater than the threshold
            if dfA['HeadPosZA'].max() > out_threshold - 1:
                # if so, get the time when it first happens
                # create a new dataframe that only contains the rows where Z position is greater than threshold, then get the first row's time
                timeA = dfA.loc[dfA['HeadPosZA'] > out_threshold - 1, 'Time'].iloc[0]   
        elif carA_out_direction == "bottom":
            if dfA['HeadPosZA'].min() < -out_threshold + 1:
                timeA = dfA.loc[dfA['HeadPosZA'] < -out_threshold + 1, 'Time'].iloc[0]
        elif carA_out_direction == "left":
            if dfA['HeadPosXA'].min() < -out_threshold + 1:
                timeA = dfA.loc[dfA['HeadPosXA'] < -out_threshold + 1, 'Time'].iloc[0]
        elif carA_out_direction == "right":
            if dfA['HeadPosXA'].max() > out_threshold - 1:
                timeA = dfA.loc[dfA['HeadPosXA'] > out_threshold - 1, 'Time'].iloc[0]

        if carB_out_direction == "top":
            if dfB['HeadPosZB'].max() > out_threshold - 1:
                timeB = dfB.loc[dfB['HeadPosZB'] > out_threshold - 1, 'Time'].iloc[0]
        elif carB_out_direction == "bottom":
            if dfB['HeadPosZB'].min() < -out_threshold + 1:
                timeB = dfB.loc[dfB['HeadPosZB'] < -out_threshold + 1, 'Time'].iloc[0]
        elif carB_out_direction == "left":
            if dfB['HeadPosXB'].min() < -out_threshold + 1:
                timeB = dfB.loc[dfB['HeadPosXB'] < -out_threshold + 1, 'Time'].iloc[0]
        elif carB_out_direction == "right":
            if dfB['HeadPosXB'].max() > out_threshold - 1:
                timeB = dfB.loc[dfB['HeadPosXB'] > out_threshold - 1, 'Time'].iloc[0]

        timeA_list.append(timeA)
        timeB_list.append(timeB)
    
    print(timeA_list)
    print(timeB_list)

    out_first_list = []
    # compare which car is out first
    for i in range(len(timeA_list)):
        if timeA_list[i] < timeB_list[i]:
            out_first_list.append("A")
        elif timeA_list[i] > timeB_list[i]:
            out_first_list.append("B")
        else:
            out_first_list.append("same")
    print(out_first_list)
    # print the amount of A and B
    print(f"car A out first: {out_first_list.count('A')}, car B out first: {out_first_list.count('B')}")

    # print the percentage of A and B
    print(f"car A out first: {(out_first_list.count('A') / len(out_first_list))}, car B out first: {out_first_list.count('B') / len(out_first_list)}")

    # print call file path where a car is out first
    """
    for i in range(len(out_first_list)):
        if out_first_list[i] == "A":
            print(CPlist[i])
    """




In [32]:
# create a dictionary to store out direction of each cp
# key: cp number, value: out direction
cp_out_direction_A = {}
cp_out_direction_B = {}

cp_out_direction_A[1] = "left"
cp_out_direction_B[1] = "bottom"
cp_out_direction_A[2] = "left"
cp_out_direction_B[2] = "right"
cp_out_direction_A[3] = "left"
cp_out_direction_B[3] = "bottom"
cp_out_direction_A[5] = "left"
cp_out_direction_B[5] = "right"
cp_out_direction_A[6] = "right"
cp_out_direction_B[6] = "right"
cp_out_direction_A[7] = "top"
cp_out_direction_B[7] = "right"
cp_out_direction_A[8] = "left"
cp_out_direction_B[8] = "right"

my_cp = 7
bound = 25
CPlist_NYC, CPlist_ITH, CPlist_ISR = SetCPList(my_cp)
FirstOutAnalysis(CPlist_NYC, cp_out_direction_A[my_cp], cp_out_direction_B[my_cp], bound)
FirstOutAnalysis(CPlist_ISR, cp_out_direction_A[my_cp], cp_out_direction_B[my_cp], bound)

got 35 csv from CP7 in NYC with validity: True 
got 10 csv from CP7 in ITH with validity: True 
got 31 csv from CP7 in ISR with validity: True 
[99999, 334, 303, 209, 99999, 364, 99999, 163, 99999, 310, 262, 99999, 218, 273, 226, 217, 99999, 293, 165, 539, 292, 309, 99999, 351, 351, 99999, 99999, 208, 432, 99999, 482, 148, 714, 329, 216]
[83, 214, 147, 341, 330, 670, 214, 99999, 411, 99999, 99999, 213, 99999, 202, 99999, 99999, 166, 289, 99999, 380, 343, 381, 213, 99999, 389, 466, 330, 99999, 581, 206, 379, 204, 477, 99999, 99999]
['B', 'B', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'A', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'B', 'A', 'A', 'B', 'B', 'A', 'B', 'A', 'A']
car A out first: 18, car B out first: 17
car A out first: 0.5142857142857142, car B out first: 0.4857142857142857
[167, 414, 244, 99999, 255, 225, 237, 259, 150, 229, 278, 350, 261, 99999, 340, 196, 274, 320, 99999, 99999, 99999, 177, 233, 191, 273, 191, 166, 99999, 136, 184, 431]
[259,